In [0]:
# df = spark.read.table('main_prod.datascience_scratchpad.traj_data')
df = spark.read.parquet("/Volumes/main_prod/datascience_scratchpad/jatin/trajcl_exp/usa/last_20_days")
display(df)

In [0]:
# from pyspark.sql.functions import col
# userid = "5499415"
# candidate_df = df.filter(col("traj_id").contains("{}_".format(userid)))
# display(candidate_df)

In [0]:
!pip install folium
import folium

from folium import plugins

def display_traj(lon_lat_list,time_indices=None):
    coordinates = [[x[1],x[0]] for x in lon_lat_list]
    # Initialize map
    m = folium.Map(location=coordinates[0], zoom_start=15)


    # for lat,lon in pip_list:
    #     folium.Marker(
    #         location=[lat, lon],
    #         popup="Pip Location",
    #         icon=folium.Icon(color='red')  # Color can be 'red', 'blue', 'green', 'purple', etc.
    #     ).add_to(m)

    folium.Marker(
        location=coordinates[0],
        popup=time_indices[0],
        icon=folium.Icon(color='green')  # Color can be 'red', 'blue', 'green', 'purple', etc.
    ).add_to(m)

    folium.Marker(
        location=coordinates[-1],
        popup=time_indices[-1],
        icon=folium.Icon(color='red')  # Color can be 'red', 'blue', 'green', 'purple', etc.
    ).add_to(m)
    # Add markers
    for i, (lat, lon) in enumerate(coordinates[1:-1]):
        if time_indices is None:
            folium.Marker([lat, lon], popup=(lat, lon)).add_to(m)
        else:
            folium.Marker([lat, lon], popup=(i+1, time_indices[i+1], lat, lon)).add_to(m)
        

    # Draw arrows between points
    for i in range(len(coordinates) - 1):
        start = coordinates[i]
        end = coordinates[i + 1]

        # Draw the line
        line = folium.PolyLine([start, end], color="blue", weight=3, opacity=0.7).add_to(m)

        # Add directional arrow
        plugins.PolyLineTextPath(
            line,
            '➤',  # arrow symbol
            repeat=True,
            offset=7,
            attributes={'fill': 'blue', 'font-weight': 'bold', 'font-size': '16'}
            ).add_to(m)
        
        

    # Save and show the map
    m.save("map_with_arrows.html")
    return m


In [0]:
from pyspark.sql.functions import col
def get_candidate_df(userid):
    candidate_df = df.filter(col("userid") == userid)
    return candidate_df

In [0]:
display(df.where('userid=14855786'))

In [0]:
userid = 23122754
traj_dates = ["2025-09-25","2025-09-26","2025-09-23","2025-09-22"]
list_lng_lat_list = []
time_indices_list = []
candidate_df = get_candidate_df(userid)
for traj_date in traj_dates:
    lng_lat_list = candidate_df.where("traj_date = '{}'".format(traj_date)).select("wgs_seq").collect()[0][0]
    time_indices = candidate_df.where("traj_date = '{}'".format(traj_date)).select("sorted_ts").collect()[0][0]
    list_lng_lat_list.append(lng_lat_list)
    time_indices_list.append(time_indices)


In [0]:
# display(candidate_df)

In [0]:
display_traj(list_lng_lat_list[0], time_indices_list[0])

In [0]:
display_traj(list_lng_lat_list[1], time_indices_list[1])


In [0]:
display_traj(list_lng_lat_list[2], time_indices_list[2])

In [0]:
display_traj(list_lng_lat_list[3], time_indices_list[3])

In [0]:
ids_to_show = ["4474029_2020-01-20", "4474029_2023-08-27"]
list_lng_lat_list = []
for id in ids_to_show:
    lng_lat_list = candidate_df.filter(col("traj_id") == id).select("wgs_seq_filtered").collect()[0][0]
    list_lng_lat_list.append(lng_lat_list)   

In [0]:
display_traj(list_lng_lat_list[0])

In [0]:
display_traj(list_lng_lat_list[1])

In [0]:
df = spark.read.table("main_prod.datascience_scratchpad.all_traj_data")
display(df)

In [0]:
query = """
select * from main_prod.datascience_scratchpad.all_traj_data where userid = 17622592
"""
df = spark.sql(query)
display(df)